In [1]:
import tensorflow as tf
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'
import numpy as np
from model import *
from utility import *

model_name = "WGAN_for_irasutoya"

In [2]:
class Generator:
    def __init__(self):
        self.reuse = False
        self.g_bn0 = BatchNormalization(name = 'g_bn0')
        self.g_bn1 = BatchNormalization(name = 'g_bn1')
        self.g_bn2 = BatchNormalization(name = 'g_bn2')
        self.g_bn3 = BatchNormalization(name = 'g_bn3')

    def __call__(self, z):
        with tf.variable_scope('g', reuse=self.reuse):

            fc0 = full_connection_layer(z, 512*6*6, name="fc0")
            fc0 = tf.reshape(fc0, [-1, 6, 6, 512])

            batch_size = tf.shape(fc0)[0]
            deconv0 = deconv2d_layer(fc0, [batch_size, 12, 12, 256], kernel_size=5, name="deconv0")
            deconv0 = self.g_bn0(deconv0)
            deconv0 = lrelu(deconv0, leak=0.3)

            deconv1 = deconv2d_layer(deconv0, [batch_size, 24, 24, 128], kernel_size=5, name="deconv1")
            deconv1 = self.g_bn1(deconv1)
            deconv1 = lrelu(deconv1, leak=0.3)

            deconv2 = deconv2d_layer(deconv1, [batch_size, 48, 48, 64], kernel_size=5, name="deconv2")
            deconv2 = self.g_bn2(deconv2)
            deconv2 = lrelu(deconv2, leak=0.3)

            deconv3 = deconv2d_layer(deconv2, [batch_size, 96, 96, 3], kernel_size=5, name="deconv3")

            output = tf.nn.tanh(deconv3)
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return output

In [3]:
class Discriminator:
    def __init__(self):
        self.reuse = False
        self.d_bn0 = BatchNormalization(name="d_bn0")
        self.d_bn1 = BatchNormalization(name="d_bn1")
        self.d_bn2 = BatchNormalization(name="d_bn2")
        self.d_bn3 = BatchNormalization(name="d_bn3")
        self.d_bn4 = BatchNormalization(name="d_bn4")
        
    def __call__(self, x):
        with tf.variable_scope('d', reuse=self.reuse):
            x = tf.reshape(x, [-1, 96, 96, 3])

            conv1 = conv2d_layer(x, 64, kernel_size=5, name="d_conv0")
            conv1 = self.d_bn0(conv1)
            conv1 = lrelu(conv1, leak=0.3)
            
            conv2 = conv2d_layer(conv1, 128, kernel_size=5, name="d_conv1")
            conv2 = self.d_bn1(conv2)
            conv2 = lrelu(conv2, leak=0.3)

            conv3 = conv2d_layer(conv2, 256, kernel_size=5, name="d_conv2")
            conv3 = self.d_bn2(conv3)
            conv3 = lrelu(conv3, leak=0.3)

            conv4 = conv2d_layer(conv3, 512, kernel_size=5, name="d_conv3")
            conv4 = self.d_bn3(conv4)
            conv4 = lrelu(conv4, leak=0.3)
            
            disc = full_connection_layer(conv4, 1, name="disc")

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')

        return disc

In [4]:
class GAN:
    def __init__(self):
        self.batch_size = 64
        self.img_size = 96
        self.z_size = 100
        
        self.epochs = 50000
        self.epoch_saveMetrics = 1000
        self.epoch_saveSampleImg = 1000
        self.epoch_saveParamter = 5000
        self.losses = {"d_loss":[], "g_loss":[]}

        # unrolled counts
        self.steps = 5

        self.dataset = np.load("irasutoya_face_1813x96x96x3_jpg.npy")
        self.dataset = (self.dataset/255)# - 0.5

        self.X_tr = tf.placeholder(tf.float32, shape=[None, self.img_size, self.img_size, 3])
        self.z = tf.placeholder(tf.float32, [None, self.z_size])
        
        self.g = Generator()
        self.d = Discriminator()
        self.Xg = self.g(self.z)



    def loss(self):
        disc_tr = self.d(self.X_tr)
        disc_gen = self.d(self.Xg)
        
        loss_d = -tf.reduce_mean(disc_tr) + tf.reduce_mean(disc_gen)
        loss_g = -tf.reduce_mean(disc_gen)

        return loss_g, loss_d

    def train(self):
        # Optimizer

        self.L_g, self.L_d = self.loss()

        d_opt = tf.train.RMSPropOptimizer(learning_rate=5e-5)
        d_train_op = d_opt.minimize(self.L_d, var_list=self.d.variables)
        g_opt = tf.train.RMSPropOptimizer(learning_rate=5e-5)
        g_train_op = g_opt.minimize(self.L_g, var_list=self.g.variables)

        self.clip_updates = [w.assign(tf.clip_by_value(w, -0.01, 0.01)) for w in self.d.variables]

        saver = tf.train.Saver()
        
        config = tf.ConfigProto(
            gpu_options=tf.GPUOptions(
                visible_device_list= "0"
            )
        )
                
        with tf.Session(config=config) as sess:
            sess.run(tf.global_variables_initializer())

            # preparing noise vec for test
            bs = 100
            test_z = np.random.uniform(-1, 1, size=[bs, self.z_size])

            for epoch in range(self.epochs):
                for step in range(self.steps):
                    # extract images for training
                    rand_index = np.random.randint(0, self.dataset.shape[0], size=self.batch_size)
                    X_mb = self.dataset[rand_index, :].astype(np.float32)
                    X_mb = np.reshape(X_mb, [-1, 96, 96, 3])
                    z = np.random.uniform(-1, 1, size=[self.batch_size, self.z_size])

                    sess.run([self.clip_updates])

                    # train Discriminator
                    _, d_loss_value = sess.run([d_train_op, self.L_d], feed_dict={
                        self.X_tr: X_mb,
                        self.z:z,
                    })
         
                # extract images for training
                rand_index = np.random.randint(0, self.dataset.shape[0], size=self.batch_size)
                X_mb = self.dataset[rand_index, :].astype(np.float32)
                X_mb = np.reshape(X_mb, [-1, 96, 96, 3])
                z = np.random.uniform(-1, 1, size=[self.batch_size, self.z_size])

                # train Generator
                _, g_loss_value = sess.run([g_train_op, self.L_g], feed_dict={
                    self.X_tr: X_mb,
                    self.z:z,
                })

                # append loss value for visualizing
                self.losses["d_loss"].append(np.sum(d_loss_value))
                self.losses["g_loss"].append(np.sum(g_loss_value))
                
                # print epoch
                if epoch % 100 == 0:
                    print('epoch:{0}, d_loss:{1}, g_loss{2} '.format(epoch, d_loss_value, g_loss_value))
                
                # visualize loss
                if epoch % self.epoch_saveMetrics == 0:
                    save_metrics(model_name, self.losses, epoch)

                # visualize generated images during training
                if epoch % self.epoch_saveSampleImg == 0:
                    img = sess.run(self.Xg, feed_dict={self.z: test_z})
                    #img = (img+ 0.5)*1.0
                    save_imgs(model_name, img, name=str(epoch))

                # save model parameters 
                if epoch % self.epoch_saveParamter == 0:
                    dir_path = "model_" + model_name
                    if not os.path.isdir(dir_path):
                        os.makedirs(dir_path)

                    saver.save(sess, dir_path + "/" + str(epoch) + ".ckpt")

In [ ]:
gan = GAN()
gan.train()


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
epoch:0, d_loss:0.0033371406607329845, g_loss0.00028292727074585855 


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


[[[ 1.35598555e-02 -1.50698377e-02 -2.64549349e-02]
  [-2.64944006e-02 -3.80362719e-02  9.34520140e-02]
  [ 1.88763235e-02 -8.08126777e-02  1.25742853e-02]
  ...
  [-2.63560330e-04 -9.12168771e-02  3.68630067e-02]
  [ 9.58808511e-03 -3.73142101e-02  2.91915634e-03]
  [-2.36370992e-02 -5.20642288e-02 -2.14913357e-02]]

 [[ 9.18782782e-03  1.85968727e-02 -7.91565515e-03]
  [-1.56854969e-02  2.94159316e-02 -2.23038811e-02]
  [ 4.59615663e-02  2.88135000e-02  3.91860902e-02]
  ...
  [-4.49735038e-02  2.17335001e-02 -1.03392519e-01]
  [ 1.10578895e-01 -8.88816118e-02  1.03014857e-01]
  [ 3.93521041e-02  1.64434239e-02  1.89712998e-02]]

 [[ 7.91047607e-03 -1.74499974e-02 -3.44577730e-02]
  [-3.72592430e-03  1.38829811e-03 -2.47427300e-02]
  [ 7.60133937e-02 -3.84392515e-02  5.52408281e-04]
  ...
  [-3.38397250e-02  4.98204678e-02  1.14290370e-03]
  [ 1.07144594e-01 -1.54594213e-01 -1.97063506e-01]
  [ 1.51908780e-02  3.58005986e-02  3.84828821e-02]]

 ...

 [[ 9.60560739e-02  2.99550639e-03

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


[[[ 0.0020317  -0.02962012 -0.03424417]
  [-0.01303523 -0.0325968   0.06643081]
  [-0.02763744 -0.04777362 -0.03045256]
  ...
  [ 0.03302064 -0.0967947   0.02852669]
  [-0.10959111 -0.05094828 -0.04676832]
  [-0.05364463 -0.02081746 -0.00539581]]

 [[ 0.00087897 -0.0037421  -0.01460046]
  [-0.06440994  0.05635259 -0.05518418]
  [ 0.01502696  0.02513017  0.00351015]
  ...
  [-0.03982415 -0.05065102 -0.0502477 ]
  [ 0.08875526 -0.05974017  0.05973295]
  [ 0.09319872  0.11841264 -0.01399709]]

 [[ 0.00156506 -0.00881687 -0.0149615 ]
  [ 0.04397144 -0.02005076  0.01159794]
  [ 0.04940457 -0.0600106   0.00947891]
  ...
  [-0.07483235 -0.02914368 -0.00167155]
  [ 0.0499696  -0.17697486 -0.13274421]
  [-0.00911089 -0.06935947  0.01397759]]

 ...

 [[ 0.09890693  0.02339194  0.13568239]
  [ 0.04414891  0.11723795  0.09235217]
  [-0.03435675  0.02060095  0.03685735]
  ...
  [ 0.24247183  0.17773603 -0.20061721]
  [ 0.2869618  -0.04268476 -0.15623449]
  [-0.04971649 -0.16949715  0.11809734]]

 [

...

[[[0.6920485  0.7107026  0.6311048 ]
  [0.7688154  0.7783125  0.76935816]
  [0.8660505  0.877804   0.87494814]
  ...
  [0.92068076 0.91582996 0.9032385 ]
  [0.9096433  0.91996944 0.9152583 ]
  [0.872652   0.87889284 0.7924896 ]]

 [[0.7543821  0.7625503  0.74369764]
  [0.81802577 0.8239578  0.86935157]
  [0.8964323  0.87066805 0.90895414]
  ...
  [0.9091952  0.92930603 0.90983087]
  [0.928279   0.9237468  0.92037034]
  [0.85989594 0.8721381  0.83563095]]

 [[0.8448536  0.85028124 0.8586909 ]
  [0.87606716 0.8714359  0.8718383 ]
  [0.9051144  0.91306734 0.90995014]
  ...
  [0.8850613  0.90809906 0.88644475]
  [0.9130804  0.9173074  0.9096758 ]
  [0.8971059  0.89225644 0.8917679 ]]

 ...

 [[0.8555868  0.84512997 0.8486461 ]
  [0.8671779  0.88136035 0.8630891 ]
  [0.91454107 0.903031   0.87934095]
  ...
  [0.90082467 0.9041654  0.9179733 ]
  [0.9158914  0.9120842  0.9443192 ]
  [0.90513754 0.91674805 0.8702306 ]]

 [[0.846552   0.87530136 0.8579597 ]
  [0.8986267  0.87459594 0.8636652 ]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


[[[0.66711414 0.6825829  0.60177606]
  [0.7682351  0.73003536 0.7476289 ]
  [0.8620545  0.85472304 0.85634035]
  ...
  [0.9229074  0.9077657  0.90587515]
  [0.9168813  0.92176884 0.91546637]
  [0.88123614 0.8787429  0.8055052 ]]

 [[0.7223584  0.6942816  0.7000655 ]
  [0.8059434  0.75922894 0.8133663 ]
  [0.8945793  0.83870494 0.8914301 ]
  ...
  [0.92764604 0.9349602  0.9236601 ]
  [0.947853   0.93560874 0.9401659 ]
  [0.88592356 0.9019963  0.86331165]]

 [[0.7962966  0.7920716  0.7879977 ]
  [0.8611191  0.81964004 0.8337321 ]
  [0.9228042  0.89396876 0.88377124]
  ...
  [0.92944807 0.9259503  0.92464757]
  [0.9488479  0.93779504 0.9415554 ]
  [0.9309507  0.91986233 0.91931605]]

 ...

 [[0.86708957 0.83495003 0.84080994]
  [0.8825108  0.8958706  0.8782877 ]
  [0.9342051  0.9208816  0.9008849 ]
  ...
  [0.9035281  0.9004835  0.8964483 ]
  [0.9353789  0.89014953 0.91671467]
  [0.92173845 0.9402896  0.9016483 ]]

 [[0.8482421  0.8682025  0.8515716 ]
  [0.90267926 0.87485343 0.8570042 ]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


[[[0.69208914 0.7122843  0.6301847 ]
  [0.7835704  0.7739099  0.7808096 ]
  [0.8821409  0.8919373  0.88415676]
  ...
  [0.92369956 0.9102262  0.9023084 ]
  [0.91899514 0.925148   0.91233146]
  [0.8856076  0.8886367  0.81392795]]

 [[0.76326126 0.75739676 0.7516588 ]
  [0.8173223  0.8352662  0.8694689 ]
  [0.9059271  0.87929153 0.91383696]
  ...
  [0.911397   0.9322003  0.9107575 ]
  [0.9430706  0.91947967 0.9289815 ]
  [0.8843106  0.88636935 0.8479959 ]]

 [[0.8454527  0.85338086 0.85070676]
  [0.882395   0.8673933  0.8738368 ]
  [0.91865    0.91803575 0.918831  ]
  ...
  [0.90507656 0.9165623  0.8947113 ]
  [0.93292606 0.92303336 0.9233833 ]
  [0.9200531  0.89929974 0.90271646]]

 ...

 [[0.8706194  0.8601882  0.8679322 ]
  [0.8730834  0.88310117 0.8785538 ]
  [0.91339654 0.9091524  0.90438586]
  ...
  [0.91155046 0.9056065  0.91749173]
  [0.93997127 0.9207547  0.93031794]
  [0.9109529  0.92313004 0.90724164]]

 [[0.858065   0.8858485  0.8720867 ]
  [0.9016494  0.8837477  0.87659687]
